## Non-contextualized model

Let's train a model without using the news

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets()

#train_dataset = train_dataset.select(range(500))

Let's take just the comments

It seems that somehow it is evenly distributed...

In [2]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-cased'

device = "cuda" if torch.cuda.is_available() else "cpu"

id2label = {0: 'Not hateful', 1: 'Hateful'}
label2id = {v:k for k,v in id2label.items()}

model = BertForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=2)

model.config.id2label = id2label
model.config.label2id = label2id

#model = model.to(device)
model.train();



tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

In [3]:
def tokenize(batch, context=True, padding='max_length', truncation=True):
    """
    Apply tokenization
    
    Arguments:
    ---------
    
    use_context: boolean (default True)
        Whether to add the context to the 
    """
    
    if context:
        args = [batch['context'], batch['text']]
    else:
        args = [batch['text']]
        
    return tokenizer(*args, padding='max_length', truncation=True)

batch_size = 32
eval_batch_size = 16

my_tokenize = lambda x: tokenize(x, context=False)

train_dataset = train_dataset.map(my_tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(my_tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(my_tokenize, batched=True, batch_size=eval_batch_size)



In [4]:
tokenizer.decode(train_dataset["input_ids"][4])

'[CLS] usuario Lo único bueno del gobierno de macri fue la definición hacia massa : el politico menos confiable de la argentina.. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [5]:

def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['HATEFUL']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

In [6]:
train_dataset

Dataset({
    features: ['APPEARANCE', 'CALLS', 'CLASS', 'CRIMINAL', 'DISABLED', 'HATEFUL', 'LGBTI', 'POLITICS', 'RACISM', 'WOMEN', 'attention_mask', 'context', 'input_ids', 'labels', 'text', 'token_type_ids'],
    num_rows: 33501
})

In [7]:
dev_dataset

Dataset({
    features: ['APPEARANCE', 'CALLS', 'CLASS', 'CRIMINAL', 'DISABLED', 'HATEFUL', 'LGBTI', 'POLITICS', 'RACISM', 'WOMEN', 'attention_mask', 'context', 'input_ids', 'labels', 'text', 'token_type_ids'],
    num_rows: 8376
})

In [8]:
from hatedetection.metrics import compute_hate_metrics
from transformers import Trainer, TrainingArguments
epochs = 10

total_steps = (epochs * len(train_dataset)) // batch_size
warmup_steps = total_steps // 10
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    warmup_steps=warmup_steps,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

results = []

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()



Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [7]:
trainer.evaluate(dev_dataset)

{'eval_loss': 0.855299174785614,
 'eval_accuracy': 0.7902340019102196,
 'eval_f1': 0.6416950756881777,
 'eval_precision': 0.6337423921917316,
 'eval_recall': 0.6529725748737397,
 'eval_runtime': 35.1425,
 'eval_samples_per_second': 238.344,
 'epoch': 10.0,
 'eval_mem_cpu_alloc_delta': 187110,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 708008,
 'eval_mem_gpu_peaked_delta': 69433856}